In [12]:
import os, glob, shutil, json
from pathlib import Path
import SimpleITK as sitk
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import datetime
from pprint import pprint

sys.path.append("/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/analysis")
from revert_normalisation import get_ct_normalisation_values

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
os.environ["nnUNet_raw"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw"
os.environ["nnUNet_preprocessed"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed"
os.environ["nnUNet_results"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results"

data_root = "/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1"

submission_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission"
submission_name = f'task1_{datetime.datetime.now().strftime("%Y%m%d")}_3models_res'


# list_data_mri = sorted(glob.glob(os.path.join(data_root, '**','mr.mha'), recursive=True))
# list_data_mask = sorted(glob.glob(os.path.join(data_root, '**','mask.mha'), recursive=True))
# print("input1 ---", len(list_data_mri), list_data_mri)
# print("input2 ---", len(list_data_mask), list_data_mask)

In [ ]:
def revert_normalisation(pred_path, ct_mean, ct_std, save_path=None):
    """
    Revert the normalisation of a CT image.
    """
    if save_path is None:
        save_path = pred_path + '_revert_norm'
    os.makedirs(save_path, exist_ok=True)
    imgs = glob.glob(os.path.join(pred_path, "*.mha"))
    for img in tqdm(imgs):
        img_sitk = sitk.ReadImage(img)
        img_array = sitk.GetArrayFromImage(img_sitk)
        img_array = img_array * ct_std + ct_mean
        img_sitk_reverted = sitk.GetImageFromArray(img_array)
        img_sitk_reverted.CopyInformation(img_sitk)
        sitk.WriteImage(img_sitk_reverted, os.path.join(save_path, os.path.basename(img)))

# Dataset AB

In [16]:
config = json.load(open("config_310.json"))
print(config)

{'dataset_id': 310, 'dataset_data_name': 'synthrad2025_task1_MR_AB_res', 'dataset_target_name': 'synthrad2025_task1_CT_AB_res', 'data_root': '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB', 'preprocessing_CT': 'CT', 'preprocessing_MRI': 'MR', 'preprocessing_mask': 'no mask'}


In [17]:
# copy files to nnUNet raw data directory

dataset_id = config['dataset_id']
dataset_data_name = config['dataset_data_name']
plan = "nnUNetResEncUNetPlans_80G"


In [18]:
input_dir = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset300_synthrad2025_task1_MR_AB/imagesVal"
# create submission folder with date today
submission_name = f'task1_{datetime.datetime.now().strftime("%Y%m%d")}_3models'
output_dir = os.path.join(submission_path, submission_name, f'DatasetID{dataset_id:03d}_{dataset_data_name}')
os.makedirs(output_dir, exist_ok=True)
fold = 0

In [19]:
os.system(f"nnUNetv2_predict -d {dataset_id} -i {input_dir} -o {output_dir} -c 3d_fullres -p {plan} -tr nnUNetTrainerMRCT -f {fold}")


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

tile :  0.5
There are 27 cases in the source folder
[['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset300_synthrad2025_task1_MR_AB/imagesVal/1ABA002_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset300_synthrad2025_task1_MR_AB/imagesVal/1ABA008_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset300_synthrad2025_task1_MR_AB/imagesVal/1ABA010_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/ra

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.34it/s]


sending off prediction to background worker for resampling and export
done with 1ABA002

Predicting 1ABA008:
perform_everything_on_device: True
(69, 329, 368) -0.8477 2.645 float16
Reconstruction: MEAN


100%|████████████████████████████████████████████| 2/2 [00:00<00:00,  5.09it/s]


sending off prediction to background worker for resampling and export
done with 1ABA008

Predicting 1ABA010:
perform_everything_on_device: True
(72, 349, 470) -0.8154 2.459 float16
Reconstruction: MEAN


...

100%|████████████████████████████████████████████| 4/4 [00:00<00:00,  5.15it/s]


sending off prediction to background worker for resampling and export
done with 1ABC015

Predicting 1ABC145:
perform_everything_on_device: True
(88, 459, 502) -0.8867 2.451 float16
Reconstruction: MEAN


100%|████████████████████████████████████████████| 4/4 [00:00<00:00,  5.14it/s]


sending off prediction to background worker for resampling and export
done with 1ABC145
(116, 360, 475) -0.838 2.63 float16


0

In [20]:

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/{plan}.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
revert_normalisation(output_dir, ct_mean, ct_std, save_path=output_dir + "_revert_norm")
# move previous results to backup
if os.path.exists(output_dir):
    shutil.move(output_dir, output_dir + "_revert_norm/backup_normalised")

CT mean: -660.54248046875, CT std: 452.00933837890625


  0%|          | 0/27 [00:00<?, ?it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


  4%|▎         | 1/27 [00:00<00:07,  3.34it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


...

 93%|█████████▎| 25/27 [00:17<00:01,  1.26it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


 96%|█████████▋| 26/27 [00:18<00:00,  1.29it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


100%|██████████| 27/27 [00:19<00:00,  1.39it/s]


# Dataset HN

In [21]:
# copy files to nnUNet raw data directory
config = json.load(open("config_312.json"))
pprint(config)
dataset_id = config['dataset_id']
dataset_data_name = config['dataset_data_name']
plan = "nnUNetResEncUNetMPlans"

{'configuration': '3d_fullres',
 'data_root': '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/HN',
 'dataset_data_name': 'synthrad2025_task1_MR_HN_res',
 'dataset_id': 312,
 'dataset_plan': 'nnUNetPlannerResEnc(M/L/XL)',
 'dataset_target_name': 'synthrad2025_task1_CT_HN_res',
 'fold': 0,
 'preprocessing_CT': 'CT',
 'preprocessing_MRI': 'MR',
 'preprocessing_mask': 'no mask',
 'region': 'HN',
 'trainer': 'nnUNetTrainerMRCT'}


In [22]:
input_dir = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset302_synthrad2025_task1_MR_HN/imagesVal"

output_dir = os.path.join(submission_path, submission_name, f'DatasetID{dataset_id:03d}_{dataset_data_name}')
os.makedirs(output_dir, exist_ok=True)
fold = 0
os.system(f"nnUNetv2_predict -d {dataset_id} -i {input_dir} -o {output_dir} -c 3d_fullres -p {plan} -tr nnUNetTrainerMRCT -f {fold}")


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

tile :  0.5
There are 34 cases in the source folder
[['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset302_synthrad2025_task1_MR_HN/imagesVal/1HNA002_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset302_synthrad2025_task1_MR_HN/imagesVal/1HNA005_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset302_synthrad2025_task1_MR_HN/imagesVal/1HNA017_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/ra

100%|██████████████████████████████████████████| 72/72 [00:01<00:00, 36.21it/s]


sending off prediction to background worker for resampling and export
done with 1HNA002

Predicting 1HNA005:
perform_everything_on_device: True
(138, 381, 519) -0.7163 4.176 float16
Reconstruction: MEAN


100%|██████████████████████████████████████████| 72/72 [00:01<00:00, 43.77it/s]


sending off prediction to background worker for resampling and export
done with 1HNA005

Predicting 1HNA017:
perform_everything_on_device: True
(115, 383, 499) -0.6875 4.2 float16
Reconstruction: MEAN


...

100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 45.38it/s]


sending off prediction to background worker for resampling and export
done with 1HND085

Predicting 1HND091:
perform_everything_on_device: True
(88, 271, 279) -0.621 4.188 float16
Reconstruction: MEAN


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 45.40it/s]


sending off prediction to background worker for resampling and export
done with 1HND091
(88, 268, 276) -0.605 4.258 float16


0

In [23]:

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/{plan}.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
revert_normalisation(output_dir, ct_mean, ct_std, save_path=output_dir + "_revert_norm")
# move previous results to backup
if os.path.exists(output_dir):
    shutil.move(output_dir, output_dir + "_revert_norm/backup_normalised")

CT mean: -756.8412475585938, CT std: 460.7137756347656


  0%|          | 0/34 [00:00<?, ?it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


  3%|▎         | 1/34 [00:01<00:34,  1.03s/it]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


...

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


 97%|█████████▋| 33/34 [00:16<00:00,  3.83it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


100%|██████████| 34/34 [00:17<00:00,  1.98it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


# Dataset TH

In [24]:
# copy files to nnUNet raw data directory
config = json.load(open("config_314.json"))
pprint(config)
dataset_id = config['dataset_id']
dataset_data_name = config['dataset_data_name']
plan = "nnUNetResEncUNetMPlans"


{'configuration': '3d_fullres',
 'data_root': '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH',
 'dataset_data_name': 'synthrad2025_task1_MR_TH_res',
 'dataset_id': 314,
 'dataset_plan': 'nnUNetPlannerResEnc(M/L/XL)',
 'dataset_target_name': 'synthrad2025_task1_CT_TH_res',
 'fold': 0,
 'preprocessing_CT': 'CT',
 'preprocessing_MRI': 'MR',
 'preprocessing_mask': 'no mask',
 'region': 'TH',
 'trainer': 'nnUNetTrainerMRCT'}


In [25]:
input_dir = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset304_synthrad2025_task1_MR_TH/imagesVal"
submission_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission"
# create submission folder with date today
submission_name = f'task1_{datetime.datetime.now().strftime("%Y%m%d")}_3models'
output_dir = os.path.join(submission_path, submission_name, f'DatasetID{dataset_id:03d}_{dataset_data_name}')
os.makedirs(output_dir, exist_ok=True)
fold = 0
os.system(f"nnUNetv2_predict -d {dataset_id} -i {input_dir} -o {output_dir} -c 3d_fullres -p {plan} -tr nnUNetTrainerMRCT -f {fold}")


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

tile :  0.5
There are 28 cases in the source folder
[['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset304_synthrad2025_task1_MR_TH/imagesVal/1THA007_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset304_synthrad2025_task1_MR_TH/imagesVal/1THA008_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset304_synthrad2025_task1_MR_TH/imagesVal/1THA014_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/ra

100%|██████████████████████████████████████████| 48/48 [00:01<00:00, 33.88it/s]


sending off prediction to background worker for resampling and export
done with 1THA007

Predicting 1THA008:
perform_everything_on_device: True
(97, 342, 452) -0.649 3.125 float16
Reconstruction: MEAN


100%|██████████████████████████████████████████| 48/48 [00:01<00:00, 44.46it/s]


sending off prediction to background worker for resampling and export
done with 1THA008

Predicting 1THA014:
perform_everything_on_device: True
(97, 363, 468) -0.638 3.213 float16
Reconstruction: MEAN


...

100%|████████████████████████████████████████| 100/100 [00:02<00:00, 44.21it/s]


sending off prediction to background worker for resampling and export
done with 1THB209

Predicting 1THB219:
perform_everything_on_device: True
(120, 500, 553) -0.6533 2.994 float16
Reconstruction: MEAN


100%|████████████████████████████████████████| 120/120 [00:02<00:00, 44.25it/s]


sending off prediction to background worker for resampling and export
done with 1THB219
(130, 490, 548) -0.6377 3.068 float16


0

In [26]:

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/{plan}.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
revert_normalisation(output_dir, ct_mean, ct_std, save_path=output_dir + "_revert_norm")
# move previous results to backup
if os.path.exists(output_dir):
    shutil.move(output_dir, output_dir + "_revert_norm/backup_normalised")

CT mean: -753.1258544921875, CT std: 409.4344482421875


  0%|          | 0/28 [00:00<?, ?it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


  4%|▎         | 1/28 [00:00<00:14,  1.83it/s]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


...

 93%|█████████▎| 26/28 [00:27<00:02,  1.44s/it]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


 96%|█████████▋| 27/28 [00:28<00:01,  1.38s/it]

(1.0, 1.0, 3.0)
(1.0, 1.0, 3.0)


100%|██████████| 28/28 [00:30<00:00,  1.07s/it]


# final archive and submission

In [27]:
save_path = os.path.join(submission_path, submission_name, 'Folder')
os.makedirs(save_path, exist_ok=True)

for file in glob.glob(os.path.join(submission_path, submission_name, "*_revert_norm", "*.mha")):
    new_file = os.path.join(save_path, "sct_" + os.path.basename(file))
    shutil.copyfile(file, new_file)
# zip folder
shutil.make_archive(save_path, 'zip', save_path)


'/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/task1_20250722_3models/Folder.zip'

In [28]:
print("Copying the line below to the local machine to download the zip file:")
print(f'rsync -avz xin015@virga.hpc.csiro.au:{save_path}.zip ./synthrad2025_submission')


Copying the line below to the local machine to download the zip file:
rsync -avz xin015@virga.hpc.csiro.au:/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/task1_20250722_3models/Folder.zip ./synthrad2025_submission
